In [1]:
%config InlineBackend.figure_formats = ['svg']
import quimb as qu
import quimb.tensor as qtn
import cotengra as ctg
import quf
import numpy as np
import torch
import algo_cooling as algo
import itertools
import algo_bp
import gen_loop as lg
import gen_loop_tn as lg_tn


In [2]:
opt = algo.opt_(progbar=True)

In [3]:
from quimb.tensor.belief_propagation.bp_common import (
    BeliefPropagationCommon,
    combine_local_contractions,
    create_lazy_community_edge_map,
)
from quimb.tensor.belief_propagation.l1bp import (
    contract_l1bp, L1BP,
)
from quimb.utils import oset

In [4]:
import torch
to_backend = algo.backend_torch(device = "cuda", dtype = torch.complex128, requires_grad=False)
to_backend_ = algo.backend_numpy( dtype = "complex128")

opt = algo.opt_(progbar=False)

In [5]:
# Define <peps | peps> as an easy problem for bp convergence:
Lx, Ly, bnd = (3, 3, 3)
peps = qtn.PEPS.rand(Lx=Lx, Ly=Ly, bond_dim=bnd, seed=12, dtype="complex128")
peps.apply_to_arrays(to_backend)
tn = peps.H & peps
fix = { f"I{i},{j}":(i,j) for i,j in itertools.product(range(peps.Lx), range(peps.Ly)) }


In [6]:
exact = tn.contract(all, optimize="auto-hq")
tn /= exact
exact = 1

In [7]:
bp = L1BP(tn, site_tags=peps.site_tags, damping=0.0, update="parallel")
bp.run(progbar=True, diis=True, tol = 1.e-11, max_iterations=2000)

mantissa, norm_exponent = bp.contract(strip_exponent=True)
est_norm = complex(mantissa * 10**norm_exponent)
print(est_norm)

max|dM|=3.92e-12 nconv: 3/3 : : 76it [00:00, 104.99it/s] 

(1.0605371045499572+5.5441518643728337e-17j)


In [8]:
bp = algo_bp.L1BP(tn, site_tags=peps.site_tags, damping=0.0, update="parallel")
bp.run(progbar=True, tol = 1.e-11, max_iterations=4000, diis=True)

mantissa, norm_exponent = bp.contract(strip_exponent=True)
est_norm = complex(mantissa * 10**norm_exponent)
print(est_norm)

max|dM|=3.92e-12 nconv: 3/3 : : 76it [00:00, 107.01it/s] 

(1.0605371045499572+5.5441518643728337e-17j)


In [9]:
res = algo_bp.run_bp(tn, peps.site_tags, normalize=True, print_=True,
                    progbar=True, tol=1.e-9, max_iterations=800, 
                    update='parallel', damping=0.00, diis=True)

bp = res["bp"]
tn = res["tn"]
print("bp: zeroth", res["norm"], tn.exponent)


bp runs with the parallel update ~ iter = 800


max|dM|=6.37e-10 nconv: 2/2 : : 46it [00:00, 100.33it/s]


messages normalized and projects are being cal
tn is being normalized w.r.t bp massages
bp: zeroth tensor(1.0605+4.6669e-17j, device='cuda:0', dtype=torch.complex128) tensor(0.0255+1.9994e-17j, device='cuda:0', dtype=torch.complex128)


In [10]:
#all edges in tn:
edges = [tuple(sorted(i)) for i in bp.edges.keys()]
max_edges = len(edges)
print("number of edges:", max_edges)


number of edges: 12


In [11]:
val_l = []
for count, loop_edges in enumerate(range(max_edges+1)):
    # loops = algo_bp.combine_elements(edges, loop_edges, sites = [])
    loops = lg.loop_gen_local(bp, circum=count, tags_cluster=[], sites=[], 
                               circum_g=0, tn_flat=peps, site_tags=peps.site_tags)

    res = lg_tn.bp_excited_loops_(tn, bp, loops, contract_=True, opt=opt, progbar=False, simplify=False)

    # for i in range(len(res["tn_l"])):
    #     res["tn_l"][i].draw([ "proj", "Mg"],     node_hatch={tag: '////' for tag in [ "Me", "Mg"]}, highlight_inds=res["inds_excit"][i], highlight_inds_color="darkred",  
    #         edge_alpha=1.0, edge_scale=1.20, fix=fix, node_outline_darkness=0.10, node_outline_size=1.2,     edge_color='darkred',
    #              show_tags=False, legend=False,node_scale=1.5,figsize=(8,8), return_fig=False)


    val_l.append(sum(res["excited"])) # *10**tn.exponent
    print(loop_edges, complex(sum(res["excited"])), complex(sum(val_l)), complex((exact - sum(val_l))/exact) )
    print("\n")

0 (1.0605371045499594+2.9508557816897966e-17j) (1.0605371045499594+2.9508557816897966e-17j) (-0.06053710454995942-2.9508557816897966e-17j)


1 0j (1.0605371045499594+2.9508557816897966e-17j) (-0.06053710454995942-2.9508557816897966e-17j)


2 0j (1.0605371045499594+2.9508557816897966e-17j) (-0.06053710454995942-2.9508557816897966e-17j)


3 0j (1.0605371045499594+2.9508557816897966e-17j) (-0.06053710454995942-2.9508557816897966e-17j)


4 (-0.01907044545236023+7.400844094204158e-18j) (1.0414666590975992+3.690940191110212e-17j) (-0.04146665909759917-3.690940191110212e-17j)


5 0j (1.0414666590975992+3.690940191110212e-17j) (-0.04146665909759917-3.690940191110212e-17j)


6 (-0.011181551277790141+6.925437084786616e-19j) (1.030285107819809+3.760194561958078e-17j) (-0.030285107819808976-3.760194561958078e-17j)


7 (0.010442576484249603-3.631400897541164e-17j) (1.0407276843040585+1.2879366441691427e-18j) (-0.04072768430405849-1.2879366441691427e-18j)


8 (-0.0033691839160620387+5.82224082081495